In [1]:
!pip -q install -U pip setuptools wheel
!pip -q install "pycaret==3.3.2" "gradio>=4.44,<5" "huggingface_hub<0.26.0"

import pycaret, gradio
print("PyCaret:", pycaret.__version__)
print("Gradio:", gradio.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.2 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
google-adk 1.14.1 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which

In [4]:
import pandas as pd
import gradio as gr
from pycaret.regression import load_model, predict_model

# Load the saved model
MODEL_PATH = "/kaggle/input/pycaret-models-prachi/regression_mpg_final"  # adjust if needed
reg = load_model(MODEL_PATH)

# Feature list from Auto MPG
feature_cols = [
    "cylinders", "displacement", "horsepower",
    "weight", "acceleration", "model_year", "origin"
]

# Default values for UI
defaults = dict(
    cylinders=4, displacement=120.0, horsepower=90.0,
    weight=2500.0, acceleration=15.0,
    model_year=76, origin="usa"
)

# Build inputs
inputs = [
    gr.Slider(3, 8, step=1, value=defaults["cylinders"], label="cylinders"),
    gr.Number(value=defaults["displacement"], label="displacement"),
    gr.Number(value=defaults["horsepower"], label="horsepower"),
    gr.Number(value=defaults["weight"], label="weight"),
    gr.Number(value=defaults["acceleration"], label="acceleration"),
    gr.Slider(70, 82, step=1, value=defaults["model_year"], label="model_year"),
    gr.Dropdown(choices=["usa","europe","japan"], value=defaults["origin"], label="origin"),
]

# Prediction function (use prediction_label column)
def predict_mpg(*vals):
    row = pd.DataFrame([dict(zip(feature_cols, vals))])
    out = predict_model(reg, data=row)
    # PyCaret regression outputs 'prediction_label'
    return float(out["prediction_label"].iloc[0])

# Launch Gradio app
demo = gr.Interface(
    fn=predict_mpg,
    inputs=inputs,
    outputs=gr.Number(label="Predicted mpg"),
    title="Auto MPG — Regression Demo",
    description="Interactively predict car fuel efficiency using your trained PyCaret model."
)

demo.launch(share=True)


Transformation Pipeline and Model Successfully Loaded
Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://972623de5147c9aab7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
